# SQL Aggregation

In [4]:
from mysql.connector import connect

connection = connect(
    host="localhost",
    user="root",
    password="root",
    database="debi3"
)

cursor = connection.cursor()

In [5]:
# Create tables for JOIN demonstration

# Create departments table
cursor.execute("""
CREATE TABLE IF NOT EXISTS departments (
    dept_id INT PRIMARY KEY,
    dept_name VARCHAR(50),
    location VARCHAR(50)
)
""")

# Create employees table with department reference
cursor.execute("""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INT PRIMARY KEY,
    emp_name VARCHAR(50),
    salary DECIMAL(10,2),
    dept_id INT,
    hire_date DATE
)
""")

# Create projects table
cursor.execute("""
CREATE TABLE IF NOT EXISTS projects (
    project_id INT PRIMARY KEY,
    project_name VARCHAR(100),
    budget DECIMAL(12,2),
    dept_id INT
)
""")

# Create employee_projects table (many-to-many relationship)
cursor.execute("""
CREATE TABLE IF NOT EXISTS employee_projects (
    emp_id INT,
    project_id INT,
    role VARCHAR(50),
    PRIMARY KEY (emp_id, project_id)
)
""")

connection.commit()
print("Tables created successfully!")

Tables created successfully!


In [6]:
# Insert sample data into departments table
departments_data = [
    (1, 'Human Resources', 'New York'),
    (2, 'Engineering', 'San Francisco'),
    (3, 'Marketing', 'Chicago'),
    (4, 'Finance', 'New York'),
    (5, 'Research', 'Boston')
]

cursor.executemany("INSERT IGNORE INTO departments (dept_id, dept_name, location) VALUES (%s, %s, %s)", departments_data)
connection.commit()
print("Departments data inserted!")

Departments data inserted!


In [7]:
# Insert sample data into employees table
employees_data = [
    (101, 'Alice Johnson', 75000.00, 1, '2022-01-15'),
    (102, 'Bob Smith', 85000.00, 2, '2021-06-10'),
    (103, 'Charlie Brown', 65000.00, 3, '2023-03-20'),
    (104, 'Diana Prince', 90000.00, 2, '2020-09-05'),
    (105, 'Eve Adams', 70000.00, 4, '2022-11-12'),
    (106, 'Frank Miller', 95000.00, 2, '2019-04-18'),
    (107, 'Grace Lee', 60000.00, 3, '2023-07-01'),
    (108, 'Henry Wilson', 80000.00, 1, '2021-12-03'),
    (109, 'Ivy Chen', 120000.00, None, '2023-01-10')  # Employee without department
]

cursor.executemany("INSERT IGNORE INTO employees (emp_id, emp_name, salary, dept_id, hire_date) VALUES (%s, %s, %s, %s, %s)", employees_data)
connection.commit()
print("Employees data inserted!")

Employees data inserted!


In [8]:
# Insert sample data into projects table
projects_data = [
    (201, 'Website Redesign', 150000.00, 2),
    (202, 'HR System Upgrade', 80000.00, 1),
    (203, 'Marketing Campaign', 120000.00, 3),
    (204, 'Budget Analysis Tool', 95000.00, 4),
    (205, 'Mobile App Development', 200000.00, 2),
    (206, 'Data Analytics Platform', 250000.00, 5),
    (207, 'Social Media Strategy', 75000.00, 3),
    (208, 'Legacy System Migration', 300000.00, None)  # Project without assigned department
]

cursor.executemany("INSERT IGNORE INTO projects (project_id, project_name, budget, dept_id) VALUES (%s, %s, %s, %s)", projects_data)
connection.commit()
print("Projects data inserted!")

Projects data inserted!


In [9]:
# Insert sample data into employee_projects table
employee_projects_data = [
    (102, 201, 'Lead Developer'),
    (104, 201, 'Senior Developer'),
    (106, 201, 'Technical Lead'),
    (101, 202, 'Project Manager'),
    (108, 202, 'Business Analyst'),
    (103, 203, 'Marketing Specialist'),
    (107, 203, 'Content Creator'),
    (105, 204, 'Financial Analyst'),
    (102, 205, 'Mobile Developer'),
    (104, 205, 'Backend Developer'),
    (109, 206, 'Data Scientist'),
    (103, 207, 'Social Media Manager')
]

cursor.executemany("INSERT IGNORE INTO employee_projects (emp_id, project_id, role) VALUES (%s, %s, %s)", employee_projects_data)
connection.commit()
print("Employee-Projects relationships inserted!")

Employee-Projects relationships inserted!


In [12]:
# SUM , MAX, MIN
cursor.execute(""" 
SELECT 
    SUM(e.salary),
    MAX(e.salary),
    MIN(e.salary),
    AVG(e.salary)     
FROM 
    employees e
""")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
print(data)

['SUM(e.salary)', 'MAX(e.salary)', 'MIN(e.salary)', 'AVG(e.salary)']
[(Decimal('740000.00'), Decimal('120000.00'), Decimal('60000.00'), Decimal('82222.222222'))]


In [14]:
# COUNT
cursor.execute(""" 
SELECT 
    COUNT(e.emp_id)     
FROM 
    employees e
""")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
print(data)

['COUNT(e.emp_id)']
[(9,)]


In [29]:
# COUNT in certain department "Engineeering Department"
cursor.execute(""" 
SELECT 
    COUNT(e.emp_id)     
FROM 
    employees e
WHERE
    e.dept_id = 2
""")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
print(data)

['COUNT(e.emp_id)']
[(3,)]


# Group By

In [ ]:
cursor.execute(""" 
SELECT 
    e.dept_id,
    COUNT(e.emp_id),
    AVG (e.salary)   
FROM 
    employees e
WHERE 
    e.dept_id IS NOT NULL
GROUP BY
    e.dept_id
""")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
for row in data:
    print(row)

['dept_id', 'COUNT(e.emp_id)', 'AVG (e.salary)']
(1, 2, Decimal('77500.000000'))
(2, 3, Decimal('90000.000000'))
(3, 2, Decimal('62500.000000'))
(4, 1, Decimal('70000.000000'))


In [32]:
cursor.execute(""" 
SELECT 
    e.dept_id,
    COUNT(e.emp_id),
    AVG (e.salary)   
FROM 
    employees e
WHERE 
    e.dept_id IS NOT NULL
GROUP BY
    e.dept_id
HAVING
    COUNT(e.emp_id) > 2 AND AVG(e.salary) > 50000
""")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
for row in data:
    print(row)

['dept_id', 'COUNT(e.emp_id)', 'AVG (e.salary)']
(2, 3, Decimal('90000.000000'))


# Views

In [37]:
cursor.execute(""" 
CREATE VIEW dept_summary_view AS
SELECT 
    e.dept_id,
    d.dept_name,
    COUNT(e.emp_id),
    AVG (e.salary)   
FROM 
    employees e JOIN departments d ON e.dept_id = d.dept_id
WHERE 
    e.dept_id IS NOT NULL
GROUP BY
    e.dept_id
""")

In [38]:
# List the views
cursor.execute("SHOW FULL TABLES WHERE TABLE_TYPE LIKE 'VIEW'")
views = cursor.fetchall()
for view in views:
    print(view)

('dept_summary', 'VIEW')
('dept_summary_view', 'VIEW')


In [35]:
# List data from the view
cursor.execute("SELECT * FROM dept_summary")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
for row in data:
    print(row)


['dept_id', 'COUNT(e.emp_id)', 'AVG (e.salary)']
(1, 2, Decimal('77500.000000'))
(2, 3, Decimal('90000.000000'))
(3, 2, Decimal('62500.000000'))
(4, 1, Decimal('70000.000000'))


In [39]:
# List data from the view
cursor.execute("SELECT * FROM dept_summary_view")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
for row in data:
    print(row)

['dept_id', 'dept_name', 'COUNT(e.emp_id)', 'AVG (e.salary)']
(1, 'Human Resources', 2, Decimal('77500.000000'))
(2, 'Engineering', 3, Decimal('90000.000000'))
(3, 'Marketing', 2, Decimal('62500.000000'))
(4, 'Finance', 1, Decimal('70000.000000'))


In [40]:
cursor.execute(""" 
CREATE OR REPLACE VIEW dept_summary_view AS
SELECT 
    e.dept_id,
    d.dept_name,
    COUNT(e.emp_id),
    AVG (e.salary),
    MIN(e.salary),
    MAX(e.salary)
FROM 
    employees e JOIN departments d ON e.dept_id = d.dept_id
WHERE 
    e.dept_id IS NOT NULL
GROUP BY
    e.dept_id
""")

In [41]:
# List data from the view
cursor.execute("SELECT * FROM dept_summary_view")
columns = [column[0] for column in cursor.description]
print(columns)
data = cursor.fetchall()
for row in data:
    print(row)

['dept_id', 'dept_name', 'COUNT(e.emp_id)', 'AVG (e.salary)', 'MIN(e.salary)', 'MAX(e.salary)']
(1, 'Human Resources', 2, Decimal('77500.000000'), Decimal('75000.00'), Decimal('80000.00'))
(2, 'Engineering', 3, Decimal('90000.000000'), Decimal('85000.00'), Decimal('95000.00'))
(3, 'Marketing', 2, Decimal('62500.000000'), Decimal('60000.00'), Decimal('65000.00'))
(4, 'Finance', 1, Decimal('70000.000000'), Decimal('70000.00'), Decimal('70000.00'))


In [42]:
cursor.execute("""
    DROP VIEW IF EXISTS dept_summary_view
""")

cursor.execute("""
    SHOW FULL TABLES WHERE TABLE_TYPE LIKE 'VIEW'
""")

print("Views after deletion:")
data = cursor.fetchall()
for row in data:
    print(row)


Views after deletion:
('dept_summary', 'VIEW')


# SQL over cloud

In [ ]:
!pip install "cloud-sql-python-connector[pymysql]"

In [4]:
from google.cloud.sql.connector import Connector
import sqlalchemy

# Initialize Connector object
connector = Connector()

# Initialize SQLAlchemy connection pool with Connector
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=lambda: connector.connect(
        "graphite-hook-438709-m2:europe-west1:debisql",  # Your Cloud SQL instance connection name
        "pymysql",
        user="iadel",
        password="root",
        db="debi3"
    ),
)

# Example usage (e.g., with SQLAlchemy)
with pool.connect() as conn:
    result = conn.execute(sqlalchemy.text("SELECT * FROM users"))
    print(result.fetchall())

# Close the connector when done
# connector.close()

[(1, 'Ahmed'), (2, 'Islam')]


In [16]:
# Create tables over the cloud SQL instance

with pool.connect() as conn:
    # Create employees table with department reference
    conn.execute(sqlalchemy.text("""
    CREATE TABLE IF NOT EXISTS employees (
        emp_id INT PRIMARY KEY,
        emp_name VARCHAR(50),
        salary DECIMAL(10,2),
        dept_id INT,
        hire_date DATE
    )
    """))

print("Tables created successfully!")

# Insert sample data into employees table using SQLAlchemy style parameters
employees_data = [
    {'emp_id': 101, 'emp_name': 'Alice Johnson', 'salary': 75000.00, 'dept_id': 1, 'hire_date': '2022-01-15'},
    {'emp_id': 102, 'emp_name': 'Bob Smith', 'salary': 85000.00, 'dept_id': 2, 'hire_date': '2021-06-10'},
    {'emp_id': 103, 'emp_name': 'Charlie Brown', 'salary': 65000.00, 'dept_id': 3, 'hire_date': '2023-03-20'},
    {'emp_id': 104, 'emp_name': 'Diana Prince', 'salary': 90000.00, 'dept_id': 2, 'hire_date': '2020-09-05'},
    {'emp_id': 105, 'emp_name': 'Eve Adams', 'salary': 70000.00, 'dept_id': 4, 'hire_date': '2022-11-12'},
    {'emp_id': 106, 'emp_name': 'Frank Miller', 'salary': 95000.00, 'dept_id': 2, 'hire_date': '2019-04-18'},
    {'emp_id': 107, 'emp_name': 'Grace Lee', 'salary': 60000.00, 'dept_id': 3, 'hire_date': '2023-07-01'},
    {'emp_id': 108, 'emp_name': 'Henry Wilson', 'salary': 80000.00, 'dept_id': 1, 'hire_date': '2021-12-03'},
    {'emp_id': 109, 'emp_name': 'Ivy Chen', 'salary': 120000.00, 'dept_id': None, 'hire_date': '2023-01-10'}  # Employee without department
]

with pool.connect() as conn:
    conn.execute(
        sqlalchemy.text("""
            INSERT IGNORE INTO employees (emp_id, emp_name, salary, dept_id, hire_date)
            VALUES (:emp_id, :emp_name, :salary, :dept_id, :hire_date)
        """), employees_data
    )
    conn.commit()

print("Employees data inserted!")

# connector.close()

Tables created successfully!
Employees data inserted!
Employees data inserted!
